In [ ]:
"""Your first learning agent."""
import gymnasium as gym
from matplotlib import pyplot as plt
from ray.rllib.algorithms.dqn import DQNConfig
import numpy as np

/Users/davide/miniconda3/envs/rl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-30 13:53:26,088	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-30 13:53:29,009	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# 0 - Choose an algorithm from ray.rllib.algorithms, e.g. ray.rllib.algorithms.xxx as
# xxxConfig
# 1 -  Build an agent
# 1.1 - Get the default config of xxxConfig()
# 1.2 - Examine the config by converting it to a dict via .to_dict() method
# 1.3 - Modify the config if needed, e.g. change the "num_gpus" to 0,
# 1.4 - Introduce the environment to the agent's config
# 1.5 - Build the agent
# 2 - Train the agent for one training round and get the reports
# 3 - Run a loop for nr_trainings = 50 times

# 4 - Visualize the trained agent; This is similar to running the random_agent,
# except that this time we have a trained agent
# 4.1 - Create an environment similar to the training env.
# 4.2. Let the agent choose an action;
# 4.3. and pass it to the environment
# 4.4. How much reward did you get for that action? Keep the score!
# 4.5. Repeat the 4.{2,3, 4} until the end of the episode
# 4.6. How much total reward you got? What does it mean to have large/small reward?


# 1 - Build an agent
# 1.1 - Get the default config of xxxConfig()

config = DQNConfig()
# 1.2 - Examine the config by converting it to a dict via .to_dict() method
config_as_dict = config.to_dict()

# 1.3 - Modify the config if needed, e.g. change the "num_gpus" to 0, or change the learning_rate
# (lr)
# 1.4 - Introduce the environment to the agent's config
config.environment(env="CartPole-v1").framework(
    framework="tf2", eager_tracing=True
).rollouts(num_rollout_workers=4, num_envs_per_worker=2).evaluation(
    evaluation_config={"explore": False},
    evaluation_duration=10,
    evaluation_interval=1,
    evaluation_duration_unit="episodes",
)

In [ ]:
def interact(n_samples):

    counter = 0    
    while counter < n_samples:
        
        env = gym.make("CartPole-v1", render_mode="rgb_array")
        s, _ = env.reset()
        done = False  # pylint: disable=invalid-name
        cumulative_reward = 0  # pylint: disable=invalid-name

        while not done:
            # 4.2. Let the agent choose an action;
            a = agent.compute_single_action(observation=s, explore=False)
            #print(f"state{}")
            # 4.3. and pass it to the environment
            cur_output = [a] + s.tolist()
            
            s, r, terminated, truncated, info = env.step(action=a)
            cur_output = cur_output + s.tolist()
            outputs.append(cur_output)

            # 4.4. How much reward did you get for that action? Keep the score!
            cumulative_reward += r
            done = terminated or truncated
            
            if counter % 50 == 0:
                print(f"Sample {counter}")
            counter += 1

            if counter >= n_samples:
                break
            

In [13]:
# 1.5 - Build the agent from the config with .build()

agent = config.build()
# 2 - Train the agent for one training round with .train and get the reports
# reports = agent.train()
# print(reports)

training_levels = [1, 4, 15, 40, 40]
n_samples = 1000
#outputs = np.zeros((len(training_levels*n_samples), 9))
outputs = []

for training_level in training_levels:
    print(f"Training level: {training_level}")

    # 3 - Run a loop for nr_trainings = 50 times # pylint: disable=invalid-name
    mean_rewards = []

    for _ in range(training_level):
        agent.train()
        # print("total reward:", agent.evaluate()["env_runners"]["episode_reward_mean"])
        
    interact(n_samples)
        

KeyboardInterrupt: 

In [12]:
np.array(outputs).shape

(100, 9)

In [ ]:
# plot the mean rewards
plt.plot(mean_rewards)
plt.xlabel("Training rounds")
plt.ylabel("Mean reward")
plt.title("Mean reward vs. training rounds")
# save the plot
# plt.savefig("mean_reward_vs_training_rounds_2.png")
# plt.close()